In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
# patient list

patient_df = pd.read_excel('/Users/yena/Desktop/SCH/척척석사/EMR related/낙상데이터/천안/(천안병원)간호간병통합병동 낙상대상자 리스트(2017-2021.10).xlsx', index_col =0)
patient_list = patient_df['등록번호'].astype(str).unique()  # 등록번호 unique -> 228명 확인
fall_date = patient_df[['등록번호', '낙상발생일시']]

duration=[]

for i in fall_date['낙상발생일시'].tolist():
    i = str(i)
    i = i.split(' ')[0]
    duration.append(i)

fall_date['낙상발생일시'] = duration



In [4]:
# str_to_dt 함수

import datetime

def str_to_dt(x):
    x = str(x)
    d = int(x[0:4])
    y = int(x[4:6])
    z = int(x[6:8])
    # hours = int(x[8:10])
    # mins = int(x[10:12])
    # secs = int(x[12:])
    # str_data = (d+','+y+','+z+','+hours+','+mins+','+secs)
    new_time = datetime.datetime(d,y,z)
    new_time_no = new_time.replace(microsecond = 0)
    
    return new_time_no

In [5]:
def clean_str(x):
    x = x.replace('-','')
    x = str_to_dt(x)
    return x

In [6]:
def dash_clear(x):
    x = str(x)
    x = x.split(' ')[0]
    x = x.replace('-','')
    x = str_to_dt(x)
    
    return x

In [3]:
sore = pd.read_excel('/Users/yena/Desktop/SCH/척척석사/EMR related/낙상데이터/천안/(비식별화)욕창_201707_202111.xlsx')

In [7]:
len(sore['등록번호'].unique())

17531

In [8]:
sore['등록번호'] = sore['등록번호'].astype(str)
sore = sore[['등록번호', '도구명', '상세내역명', '평가일자']]
sore = sore[sore['등록번호'].isin(set(patient_list))]
# sore = sore[['등록번호', '도구명','상세내역명', '평가일자']]

len(sore['등록번호'].unique())

74

In [9]:
sore['평가일자'] = sore['평가일자'].astype(str)
fall_date = fall_date.astype(str)
sore = sore.merge(fall_date, on=['등록번호'], how='inner')
sore = sore.drop_duplicates()
len(sore['등록번호'].unique())

74

In [10]:
# sore['평가일자'] = sore['평가일자'].apply(str_to_dt)
sore['낙상발생일시'] = sore['낙상발생일시'].apply(dash_clear)

In [11]:
sore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9070 entries, 0 to 9069
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   등록번호    9070 non-null   object        
 1   도구명     9070 non-null   object        
 2   상세내역명   9070 non-null   object        
 3   평가일자    9070 non-null   object        
 4   낙상발생일시  9070 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 425.2+ KB


In [12]:
sore.head(2)

,등록번호,도구명,상세내역명,평가일자,낙상발생일시
0,1393198,영양상태,부적절함(부족),20170702,2017-07-02
1,1393198,영양상태,부적절함(부족),20170702,2017-07-10


In [13]:
sore['평가일자'] = sore['평가일자'].apply(str_to_dt)

In [14]:
sore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9070 entries, 0 to 9069
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   등록번호    9070 non-null   object        
 1   도구명     9070 non-null   object        
 2   상세내역명   9070 non-null   object        
 3   평가일자    9070 non-null   datetime64[ns]
 4   낙상발생일시  9070 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 425.2+ KB


In [15]:
sore['유무'] = sore['낙상발생일시'] -  sore['평가일자']
sore['유무'] = sore['유무'].astype(str)
cond = ['0 days', '1 days', '2 days', '3 days']
sore_df = sore[sore['유무'].isin(set(cond))]
sore_df['유무'].unique()

array(['0 days', '1 days', '3 days', '2 days'], dtype=object)

In [16]:
sore_df = sore_df.astype(str)

sore_df['작업'] = sore_df['등록번호']+'/'+sore_df['낙상발생일시']+'/'+sore_df['평가일자']+'/'+sore_df['도구명'] +'/' +sore_df['상세내역명']
sore_df.head()

,등록번호,도구명,상세내역명,평가일자,낙상발생일시,유무,작업
0,1393198,영양상태,부적절함(부족),2017-07-02,2017-07-02,0 days,1393198/2017-07-02/2017-07-02/영양상태/부적절함(부족)
3,1393198,마찰과 미끄러짐,문제 없음,2017-07-02,2017-07-02,0 days,1393198/2017-07-02/2017-07-02/마찰과 미끄러짐/문제 없음
6,1393198,기동력,정상,2017-07-02,2017-07-02,0 days,1393198/2017-07-02/2017-07-02/기동력/정상
9,1393198,활동정도,정상,2017-07-02,2017-07-02,0 days,1393198/2017-07-02/2017-07-02/활동정도/정상
12,1393198,습한정도,거의 건조,2017-07-02,2017-07-02,0 days,1393198/2017-07-02/2017-07-02/습한정도/거의 건조


In [17]:
adl_detail_act = []
adl_detail_nut = []
adl_detail_mov = []

for i in sore_df['작업']:
    patient = i.split('/')[0]
    fall_dates = i.split('/')[1]
    # fall_dates = datetime.datetime.strptime(fall_dates, format_)
    dates = i.split('/')[2]
    # dates = datetime.datetime.strptime(dates, format_)
    adl = i.split('/')[3]
    detail = i.split('/')[-1]
    if adl == '활동정도':
        if  detail =='정상':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/활동정도'+'/'+'0'
            adl_detail_act.append(k)
        elif detail =='가끔보행':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/활동정도'+'/'+'1'
            adl_detail_act.append(k)
        elif detail =='의자고정':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/활동정도'+'/'+'2'
            adl_detail_act.append(k)
        elif detail =='침상안정':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/활동정도'+'/'+'3'
            adl_detail_act.append(k)
 
    elif adl =='영양상태':        
        if  detail =='양호(우수)':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/영양상태'+'/'+'0'
            adl_detail_nut.append(k)
        elif  detail =='적절함(보통)':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/영양상태'+'/'+'0'
            adl_detail_nut.append(k)
        elif  detail =='부적절함(부족)':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/영양상태'+'/'+'1'
            adl_detail_nut.append(k)
        elif  detail =='불량(매우부족)':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/영양상태'+'/'+'2'
            adl_detail_nut.append(k)

    elif adl =='기동력':       
        if  detail =='정상':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/기동력'+'/'+'0'
            adl_detail_mov.append(k)
        elif  detail =='전혀없음':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/기동력'+'/'+'0'
            adl_detail_mov.append(k)
        elif  detail =='약간제한':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/기동력'+'/'+'1'
            adl_detail_mov.append(k)
        elif  detail =='매우제한':
            k = patient +'/'+ str(fall_dates) + '/' + str(dates)+'/기동력'+'/'+'2'
            adl_detail_mov.append(k)

print(len(adl_detail_act))
print(len(adl_detail_nut))
print(len(adl_detail_mov))

72
72
72


In [18]:
new_patient_list = []
new_date_list = []
fall_date_list = []
adl_detail = []

for i in adl_detail_act:
    patient = i.split('/')[0]
    fall_= i.split('/')[1]
    dates = i.split('/')[2]
    adl_ = i.split('/')[-1]
    new_patient_list.append(patient)
    new_date_list.append(dates)
    fall_date_list.append(fall_)
    adl_detail.append(adl_)
    
act_df = pd.DataFrame({'등록번호':new_patient_list, '낙상발생일시':fall_date_list, '평가일자':new_date_list, '욕창_행동':adl_detail})
print(act_df.shape)
act_df.head()

(72, 4)


,등록번호,낙상발생일시,평가일자,욕창_행동
0,1393198,2017-07-02,2017-07-02,0
1,1393198,2017-07-10,2017-07-09,0
2,1331989,2017-09-19,2017-09-16,1
3,1331989,2017-09-19,2017-09-17,1
4,1331989,2017-09-19,2017-09-18,1


In [19]:
new_patient_list = []
new_date_list = []
fall_date_list = []
adl_detail = []

for i in adl_detail_nut:
    patient = i.split('/')[0]
    fall_= i.split('/')[1]
    dates = i.split('/')[2]
    adl_ = i.split('/')[-1]
    new_patient_list.append(patient)
    new_date_list.append(dates)
    fall_date_list.append(fall_)
    adl_detail.append(adl_)
    
nut_df = pd.DataFrame({'등록번호':new_patient_list, '낙상발생일시':fall_date_list, '평가일자':new_date_list, '욕창_영양':adl_detail})
print(nut_df.shape)
nut_df.head()

(72, 4)


,등록번호,낙상발생일시,평가일자,욕창_영양
0,1393198,2017-07-02,2017-07-02,1
1,1393198,2017-07-10,2017-07-09,1
2,1331989,2017-09-19,2017-09-16,0
3,1331989,2017-09-19,2017-09-17,0
4,1331989,2017-09-19,2017-09-18,0


In [20]:
new_patient_list = []
new_date_list = []
fall_date_list = []
adl_detail = []

for i in adl_detail_mov:
    patient = i.split('/')[0]
    fall_= i.split('/')[1]
    dates = i.split('/')[2]
    adl_ = i.split('/')[-1]
    new_patient_list.append(patient)
    new_date_list.append(dates)
    fall_date_list.append(fall_)
    adl_detail.append(adl_)
    
mov_df = pd.DataFrame({'등록번호':new_patient_list, '낙상발생일시':fall_date_list, '평가일자':new_date_list, '욕창_기동력':adl_detail})
print(mov_df.shape)
mov_df.head()

(72, 4)


,등록번호,낙상발생일시,평가일자,욕창_기동력
0,1393198,2017-07-02,2017-07-02,0
1,1393198,2017-07-10,2017-07-09,0
2,1331989,2017-09-19,2017-09-16,1
3,1331989,2017-09-19,2017-09-17,1
4,1331989,2017-09-19,2017-09-18,1


In [21]:
act_nut_patient = act_df.merge(nut_df, on=['등록번호', '낙상발생일시','평가일자'], how='outer')
act_nut_move_patient = act_nut_patient.merge(mov_df, on=['등록번호', '낙상발생일시','평가일자'], how='outer')
len(act_nut_move_patient['등록번호'].unique())

47

In [22]:
act_nut_move_patient

,등록번호,낙상발생일시,평가일자,욕창_행동,욕창_영양,욕창_기동력
0,1393198,2017-07-02,2017-07-02,0,1,0
1,1393198,2017-07-10,2017-07-09,0,1,0
2,1331989,2017-09-19,2017-09-16,1,0,1
3,1331989,2017-09-19,2017-09-17,1,0,1
4,1331989,2017-09-19,2017-09-18,1,0,1
...,...,...,...,...,...,...
67,1323071,2021-09-04,2021-09-03,3,0,2
68,1323071,2021-09-04,2021-09-04,3,0,2
69,239823,2021-07-06,2021-07-04,0,0,0
70,594511,2021-07-10,2021-07-10,0,0,0


In [23]:
act_nut_patient.to_csv('천안_욕창.csv')